# 3 Preprocessing<a id='3_preprocessing'></a>

In [1]:
# Import libraries
import os
import csv
import json
import numpy as np
import pandas as pd

## 3.1 Load Data

In [2]:
# Data directory references
intermediate_dir = r'..\data\intermediate'

In [3]:
# Load dtypes from df_clean from data wrangling step
dtypes_fn = 'clean_dtypes.json'
with open(os.path.join(intermediate_dir, dtypes_fn)) as f:
          clean_dtypes = json.load(f)
clean_dtypes

{'chargeoff_amount': 'float32',
 'chargeoff': 'int8',
 'closed_month': 'float32',
 'listing_creation_date': 'datetime64[ns]',
 'loan_origination_date': 'datetime64[ns]',
 'listing_status': 'Int16',
 'listing_amount': 'float32',
 'prosper_rating': 'object',
 'lender_yield': 'float32',
 'borrower_rate': 'float32',
 'borrower_apr': 'float32',
 'listing_term': 'Int16',
 'listing_monthly_payment': 'float32',
 'prosper_score': 'Int16',
 'income_range': 'Int16',
 'stated_monthly_income': 'float32',
 'income_verifiable': 'object',
 'employment_status_description': 'object',
 'occupation': 'object',
 'months_employed': 'Int32',
 'borrower_state': 'object',
 'prior_prosper_loans_active': 'float32',
 'prior_prosper_loans': 'float32',
 'prior_prosper_loans_principal_borrowed': 'float32',
 'prior_prosper_loans_principal_outstanding': 'float32',
 'prior_prosper_loans_balance_outstanding': 'float32',
 'prior_prosper_loans_cycles_billed': 'float32',
 'prior_prosper_loans_ontime_payments': 'float32',
 

In [4]:
clean_fn = 'listings_clean.csv'
# Pass datetime fields to parse_dates
dtype = {k: v for k, v in clean_dtypes.items() if v != 'datetime64[ns]'}
parse_dates = [k for k, v in clean_dtypes.items() if v == 'datetime64[ns]']
df_clean = pd.read_csv(os.path.join(intermediate_dir, clean_fn), dtype=dtype, parse_dates=parse_dates)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579634 entries, 0 to 579633
Data columns (total 67 columns):
 #   Column                                            Non-Null Count   Dtype         
---  ------                                            --------------   -----         
 0   listing_number                                    579634 non-null  int64         
 1   chargeoff_amount                                  579634 non-null  float32       
 2   chargeoff                                         579634 non-null  int8          
 3   closed_month                                      579634 non-null  float32       
 4   listing_creation_date                             579634 non-null  datetime64[ns]
 5   loan_origination_date                             579634 non-null  datetime64[ns]
 6   listing_status                                    579634 non-null  Int16         
 7   listing_amount                                    579634 non-null  float32       
 8   prosper_rating

In [5]:
# Drop columns identified in EDA stage
df = df_clean.drop(['listing_number', 'chargeoff_amount', 'closed_month', 'listing_creation_date', 'loan_origination_date', 
                    'listing_status', 'lender_yield', 'borrower_apr', 'listing_monthly_payment', 'stated_monthly_income',
                    'income_verifiable', 'months_employed',
                    'prior_prosper_loans_active', 'prior_prosper_loans', 'prior_prosper_loans_principal_borrowed',
                    'prior_prosper_loans_principal_outstanding', 'prior_prosper_loans_balance_outstanding',
                    'prior_prosper_loans_cycles_billed', 'prior_prosper_loans_ontime_payments',
                    'prior_prosper_loans_late_cycles', 'prior_prosper_loans_late_payments_one_month_plus',
                    'max_prior_prosper_loan', 'min_prior_prosper_loan', 'prior_prosper_loan_earliest_pay_off', 
                    'prior_prosper_loans31dpd', 'prior_prosper_loans61dpd', 'lender_indicator', 'co_borrower_application'
                   ], errors='ignore', axis=1)
df.head()

,chargeoff,listing_amount,prosper_rating,borrower_rate,listing_term,prosper_score,income_range,employment_status_description,occupation,borrower_state,...,IN01S,IN09S,IN21S,MT01S,OF01S,RE01S,RE21S,RT01S,S207S,dti_wprosper_loan
0,0,10000.0,AA,0.0869,60,10,3,Employed,Food Service Management,IN,...,5.0,0.0,25.0,1.0,1.0,28.0,6.0,7.0,-4.0,0.47
1,0,3500.0,AA,0.0869,60,10,5,Employed,Tradesman - Mechanic,CA,...,7.0,1.0,11.0,9.0,4.0,14.0,7.0,2.0,-4.0,0.10
2,0,3500.0,AA,0.0869,60,10,6,Employed,Civil Service,MD,...,4.0,3.0,14.0,1.0,3.0,13.0,13.0,7.0,-4.0,0.12
3,0,10000.0,AA,0.0869,60,10,3,Other,Other,MO,...,2.0,1.0,15.0,0.0,1.0,7.0,104.0,4.0,-4.0,0.22
4,1,15000.0,AA,0.0869,60,7,5,Employed,Other,OH,...,8.0,2.0,13.0,2.0,0.0,19.0,21.0,4.0,-4.0,0.28


## 3.2 Transform Columns

In [6]:
# Review counts of employment_status_description before
df['employment_status_description'].value_counts()

Employed         495105
Self-employed     44277
Other             39572
Full-time           647
Retired              15
Part-time            11
Not employed          7
Name: employment_status_description, dtype: int64

In [7]:
# Clean employment_status_description field
emp_dict = {'Full-time': 'Employed', 'Retired': 'Other', 'Not employed': 'Other', 'Part-time': 'Other'}
df['employment_type'] = df['employment_status_description'].map(emp_dict).fillna(df['employment_status_description'])
df['employment_type'].value_counts()

Employed         495752
Self-employed     44277
Other             39605
Name: employment_type, dtype: int64

In [8]:
# Create dummy columns
dummy_cols = ['prosper_rating', 'listing_term', 'employment_type']

df_dummy = pd.get_dummies(data=df, columns=dummy_cols, drop_first=True)

df_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579634 entries, 0 to 579633
Data columns (total 46 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   chargeoff                      579634 non-null  int8   
 1   listing_amount                 579634 non-null  float32
 2   borrower_rate                  579634 non-null  float32
 3   prosper_score                  579634 non-null  Int16  
 4   income_range                   579634 non-null  Int16  
 5   employment_status_description  579634 non-null  object 
 6   occupation                     579634 non-null  object 
 7   borrower_state                 579634 non-null  object 
 8   AT01S                          579634 non-null  float32
 9   AT02S                          579634 non-null  float32
 10  AU01S                          579634 non-null  float32
 11  G020S                          579634 non-null  float32
 12  G041S                         

In [9]:
# Drop employment_status_description columns
df_final = df_dummy.drop(['employment_status_description'], axis=1)

In [10]:
df_final.head()

,chargeoff,listing_amount,borrower_rate,prosper_score,income_range,occupation,borrower_state,AT01S,AT02S,AU01S,...,dti_wprosper_loan,prosper_rating_AA,prosper_rating_B,prosper_rating_C,prosper_rating_D,prosper_rating_E,prosper_rating_HR,listing_term_60,employment_type_Other,employment_type_Self-employed
0,0,10000.0,0.0869,10,3,Food Service Management,IN,35.0,19.0,1.0,...,0.47,1,0,0,0,0,0,1,0,0
1,0,3500.0,0.0869,10,5,Tradesman - Mechanic,CA,30.0,9.0,3.0,...,0.10,1,0,0,0,0,0,1,0,0
2,0,3500.0,0.0869,10,6,Civil Service,MD,19.0,16.0,3.0,...,0.12,1,0,0,0,0,0,1,0,0
3,0,10000.0,0.0869,10,3,Other,MO,9.0,7.0,1.0,...,0.22,1,0,0,0,0,0,1,1,0
4,1,15000.0,0.0869,7,5,Other,OH,29.0,17.0,3.0,...,0.28,1,0,0,0,0,0,1,0,0


## 3.3 Save Data

In [11]:
# Save data to train folder
final_fn = r'..\data\train\listings_final.csv'
df_final.to_csv(final_fn)

In [12]:
clean_dt = df_final.dtypes.apply(lambda x: x.name).to_dict()
dtypes_fn = r'..\data\train\final_dtypes.json'
with open(dtypes_fn, 'w') as f:
    json.dump(clean_dt, f)

**Preprocessing summary:**

*Some preprocessing steps will be deferred to the modeling stage to avoid having to save separate train and test files and also to allow for several categorical encoders to be evaluated for optimal model performance.*

- `listing_amount`, `borrower_rate`, `dti_wprosper_loan` and all credit bureau columns need to be normalized using StandardScaler.
- `prosper_score` and `income_range` need to be normalized using MinMaxScaler.
- `occupation` and `borrower_state` have high cardinality and several encoding methods will be evaluated during training.